In [3]:
import os
import json
import tkinter as tk
from tkinter import ttk, messagebox, filedialog

class ContactBookApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Contact Book")
        self.contacts = {}
        self.load_contacts()

        self.create_menu()
        self.create_toolbar()
        self.create_main_frame()
        self.create_contact_treeview()
        self.create_search_frame()

    def load_contacts(self):
        if os.path.exists("contacts.json"):
            with open("contacts.json", "r") as file:
                self.contacts = json.load(file)

    def save_contacts(self):
        with open("contacts.json", "w") as file:
            json.dump(self.contacts, file, indent=4)

    def create_menu(self):
        menubar = tk.Menu(self.root)
        filemenu = tk.Menu(menubar, tearoff=0)
        filemenu.add_command(label="Import Contacts", command=self.import_contacts)
        filemenu.add_command(label="Export Contacts", command=self.export_contacts)
        filemenu.add_separator()
        filemenu.add_command(label="Exit", command=self.root.quit)
        menubar.add_cascade(label="File", menu=filemenu)
        self.root.config(menu=menubar)

    def create_toolbar(self):
        toolbar = ttk.Frame(self.root)
        add_button = ttk.Button(toolbar, text="Add Contact", command=self.add_contact)
        add_button.pack(side=tk.LEFT, padx=5, pady=5)
        edit_button = ttk.Button(toolbar, text="Edit Contact", command=self.edit_contact)
        edit_button.pack(side=tk.LEFT, padx=5, pady=5)
        delete_button = ttk.Button(toolbar, text="Delete Contact", command=self.delete_contact)
        delete_button.pack(side=tk.LEFT, padx=5, pady=5)
        toolbar.pack(side=tk.TOP, fill=tk.X)

    def create_main_frame(self):
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

    def create_contact_treeview(self):
        self.contacts_treeview = ttk.Treeview(self.main_frame, columns=("Phone", "Email", "Address"))
        self.contacts_treeview.heading("#0", text="Name")
        self.contacts_treeview.heading("Phone", text="Phone")
        self.contacts_treeview.heading("Email", text="Email")
        self.contacts_treeview.heading("Address", text="Address")
        self.contacts_treeview.column("#0", width=150)
        self.contacts_treeview.column("Phone", width=120)
        self.contacts_treeview.column("Email", width=200)
        self.contacts_treeview.column("Address", width=300)
        self.contacts_treeview.pack(fill=tk.BOTH, expand=True)
        self.contacts_treeview.bind("<Double-1>", self.edit_contact)
        self.contacts_treeview.bind("<Delete>", self.delete_contact)
        self.populate_contact_treeview()

    def populate_contact_treeview(self):
        self.contacts_treeview.delete(*self.contacts_treeview.get_children())
        for name, info in self.contacts.items():
            self.contacts_treeview.insert("", "end", text=name, values=(info["phone"], info["email"], info["address"]))

    def create_search_frame(self):
        search_frame = ttk.Frame(self.main_frame)
        search_frame.pack(fill=tk.X, pady=(10, 0))

        ttk.Label(search_frame, text="Search:").pack(side=tk.LEFT, padx=(0, 5))
        self.search_entry = ttk.Entry(search_frame)
        self.search_entry.pack(side=tk.LEFT, fill=tk.X, expand=True)
        search_button = ttk.Button(search_frame, text="Search", command=self.search_contact)
        search_button.pack(side=tk.LEFT)

def search_contact(self):
    query = self.search_entry.get().lower()
    if query:
        results = {}
        for name, info in self.contacts.items():
            if query in name.lower() or query in info["phone"]:
                results[name] = info
        self.populate_contact_treeview(results)
    else:
        self.populate_contact_treeview(self.contacts)

    def add_contact(self):
        add_edit_window = AddEditContactWindow(self.root)
        self.root.wait_window(add_edit_window.top)
        if add_edit_window.contact_data:
            self.contacts[add_edit_window.contact_data["name"]] = add_edit_window.contact_data
            self.populate_contact_treeview()
            self.save_contacts()

    def edit_contact(self, event=None):
        selected_item = self.contacts_treeview.selection()
        if selected_item:
            name = self.contacts_treeview.item(selected_item, "text")
            add_edit_window = AddEditContactWindow(self.root, name=name, data=self.contacts[name])
            self.root.wait_window(add_edit_window.top)
            if add_edit_window.contact_data:
                del self.contacts[name]
                self.contacts[add_edit_window.contact_data["name"]] = add_edit_window.contact_data
                self.populate_contact_treeview()
                self.save_contacts()

    def delete_contact(self, event=None):
        selected_item = self.contacts_treeview.selection()
        if selected_item:
            name = self.contacts_treeview.item(selected_item, "text")
            confirm = messagebox.askyesno("Delete Contact", f"Are you sure you want to delete {name}?")
            if confirm:
                del self.contacts[name]
                self.populate_contact_treeview()
                self.save_contacts()

    def import_contacts(self):
        filename = filedialog.askopenfilename(filetypes=[("JSON files", "*.json")])
        if filename:
            with open(filename, "r") as file:
                imported_contacts = json.load(file)
                self.contacts.update(imported_contacts)
                self.populate_contact_treeview()
                self.save_contacts()

    def export_contacts(self):
        filename = filedialog.asksaveasfilename(defaultextension=".json", filetypes=[("JSON files", "*.json")])
        if filename:
            with open(filename, "w") as file:
                json.dump(self.contacts, file, indent=4)

class AddEditContactWindow:
    def __init__(self, parent, name="", data=None):
        self.top = tk.Toplevel(parent)
        self.top.title("Add/Edit Contact")
        self.contact_data = None

        ttk.Label(self.top, text="Name:").grid(row=0, column=0, padx=5, pady=5, sticky="e")
        ttk.Label(self.top, text="Phone:").grid(row=1, column=0, padx=5, pady=5, sticky="e")
        ttk.Label(self.top, text="Email:").grid(row=2, column=0, padx=5, pady=5, sticky="e")
        ttk.Label(self.top, text="Address:").grid(row=3, column=0, padx=5, pady=5, sticky="e")

        self.name_entry = ttk.Entry(self.top)
        self.name_entry.grid(row=0, column=1, padx=5, pady=5)
        self.name_entry.insert(0, name)

        self.phone_entry = ttk.Entry(self.top)
        self.phone_entry.grid(row=1, column=1, padx=5, pady=5)
        if data:
            self.phone_entry.insert(0, data["phone"])

        self.email_entry = ttk.Entry(self.top)
        self.email_entry.grid(row=2, column=1, padx=5, pady=5)
        if data:
            self.email_entry.insert(0, data["email"])

        self.address_entry = ttk.Entry(self.top)
        self.address_entry.grid(row=3, column=1, padx=5, pady=5)
        if data:
            self.address_entry.insert(0, data["address"])

        save_button = ttk.Button(self.top, text="Save", command=self.save_contact)
        save_button.grid(row=4, column=0, columnspan=2, padx=5, pady=10)

    def save_contact(self):
        name = self.name_entry.get().strip()
        phone = self.phone_entry.get().strip()
        email = self.email_entry.get().strip()
        address = self.address_entry.get().strip()
        if name and phone:
            self.contact_data = {"name": name, "phone": phone, "email": email, "address": address}
            self.top.destroy()
        else:
            messagebox.showerror("Error", "Name and phone are required.")

def main():
    root = tk.Tk()
    app = ContactBookApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\balaj\AppData\Local\Temp\ipykernel_24260\155640830.py", line 89, in search_contact
    self.populate_contact_treeview(results)
TypeError: ContactBookApp.populate_contact_treeview() takes 1 positional argument but 2 were given
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\balaj\AppData\Local\Temp\ipykernel_24260\155640830.py", line 89, in search_contact
    self.populate_contact_treeview(results)
TypeError: ContactBookApp.populate_contact_treeview() takes 1 p